### Objective: To find duplicate courses along with its main course using fuzzywuzzy logic.

In [1]:
# Import required modules
import pandas as pd
from rapidfuzz import fuzz, process
from concurrent.futures import ProcessPoolExecutor

In [2]:
# Read in the data
df = pd.read_csv("13_Feb_21_15_providers.csv").query("courseProvider=='One Education'")

# Lower case the course title
df.courseTitle = df.courseTitle.str.lower().str.strip()
df.head()

,date,courseId,courseTitle,courseLink,subtitle,courseProvider,offerPrice,originalPrice,unitSold,category,...,cpdProvider,awardingBody,qualName,isRegulated,soldOrEnq,savingsPercent,broadCategory1,broadCategory2,subCategory1,subCategory2
0,13_Feb_21,239173,hr and payroll administrator course,https://www.reed.co.uk/courses/hr-and-payroll-...,2021 Welcome Gift! | ABC Awards-Endorsed Level...,One Education,10,425,2494,[],...,CPD Accreditation Group,na,na,0,1,97,NaN,NaN,NaN,NaN
1,13_Feb_21,234929,car mechanic training,https://www.reed.co.uk/courses/car-mechanic-tr...,2021 Welcome Gift! | Accredited by IAO & CPD |...,One Education,10,425,4227,"[['Driving', 'Car maintenance'], ['Driving', '...",...,CPD Accreditation Group,na,na,0,1,97,Driving,Engineering,Car maintenance,Mechanical engineering
2,13_Feb_21,233614,estate agent diploma,https://www.reed.co.uk/courses/estate-agent-di...,2021 Welcome Gift! | ABC Awards-Endorsed Level...,One Education,18,425,2319,"[['Sales'], ['Construction', 'Estate agent'], ...",...,CPD Accreditation Group,The Quality Licence Scheme,na,0,1,95,Sales,Surveying,Sales,Estate agent
3,13_Feb_21,280114,certified counsellor - complete training (depr...,https://www.reed.co.uk/courses/certified-couns...,2021 Welcome Gift! | ABC Awards-Endorsed Level...,One Education,25,1225,523,"[['Health & care', 'Care', 'Counselling'], ['T...",...,CPD Accreditation Group,The Quality Licence Scheme,na,0,1,97,Health & care,Care,Counselling,Child Counselling
4,13_Feb_21,282714,mental health,https://www.reed.co.uk/courses/mental-health/2...,ABC Awards-Endorsed Level 5 Mental Health Awar...,One Education,10,425,222,"[['Law', 'Mental health law'], ['Health & care...",...,CPD Accreditation Group,The Quality Licence Scheme,na,0,1,97,Law,Health & care,Mental health law,Mental health nursing


In [3]:
df[df.courseTitle=="microsoft excel"]

,date,courseId,courseTitle,courseLink,subtitle,courseProvider,offerPrice,originalPrice,unitSold,category,...,cpdProvider,awardingBody,qualName,isRegulated,soldOrEnq,savingsPercent,broadCategory1,broadCategory2,subCategory1,subCategory2
67,13_Feb_21,237280,microsoft excel,https://www.reed.co.uk/courses/microsoft-excel...,2021 Welcome Gift! - IAO & CPD Accredited Leve...,One Education,10,425,1699,[],...,CPD Accreditation Group,na,na,0,1,97,NaN,NaN,NaN,NaN
485,13_Feb_21,274379,microsoft excel,https://www.reed.co.uk/courses/microsoft-excel...,ABC Awards-Endorsed Level 7 Advanced Diploma |...,One Education,10,425,34,[],...,CPD Accreditation Group,na,na,0,1,97,NaN,NaN,NaN,NaN
1354,13_Feb_21,294122,microsoft excel,https://www.reed.co.uk/courses/microsoft-excel...,6-Courses Bundle (ABC Awards-Endorsed Level 7 ...,One Education,139,1699,0,[],...,CPD Accreditation Group,na,na,0,1,91,NaN,NaN,NaN,NaN
1670,13_Feb_21,298767,microsoft excel,https://www.reed.co.uk/courses/microsoft-excel...,13-Courses' Specialised Bundle Accredited by I...,One Education,49,1899,0,[],...,CPD Accreditation Group,na,na,0,1,97,NaN,NaN,NaN,NaN


In [4]:
# Filter unique and duplicates
dfUn = df[~df.duplicated("courseTitle")]
dfDup = df[df.duplicated("courseTitle")]

In [5]:
# This function finds duplicates with corresponding main courses
def findDuplicatesWithFuzzywuzzy(text):
    """text = the course for which we are trying to find duplicates"""
    
    # This returns main course at index 0, and duplicates for rest of the indices
    mainAndDuplicates = process.extract(text, dfUn.courseTitle, score_cutoff=95, scorer=fuzz.WRatio)
    
    # This the the duplicate course df
    duplicatesDf = pd.DataFrame(mainAndDuplicates[1:], columns=["duplicates","score", "index"])
    
    # This is the main course df, for which duplicates are found
    duplicatesDf["main"] = text
    
    # Returns main course along with duplicates with similarity score
    return duplicatesDf

In [6]:
%%time
# Apply the function to find duplicates along with its main course.
"""Using multiprocessing to speed up the process. This runtime varies according to cpu config"""
with ProcessPoolExecutor() as ex:
    mainWithDuplicates = pd.concat(list(ex.map(findDuplicatesWithFuzzywuzzy, dfUn.courseTitle))).reset_index(drop=True)

CPU times: user 1.13 s, sys: 88.6 ms, total: 1.22 s
Wall time: 1.26 s


In [7]:
# Create an indicator column and sort it by its word. This is to drop duplicates
mainWithDuplicates["ind"] = mainWithDuplicates.main + " " + mainWithDuplicates.duplicates
mainWithDuplicates.ind = mainWithDuplicates["ind"].apply(lambda x: " ".join(sorted(x.split(" "))))

# Drop duplicates by "ind" column
mainWithDuplicates = mainWithDuplicates.drop_duplicates("ind")

In [8]:
# Now "game development" should be in only "main" column, but not in "duplicates" columns
display(mainWithDuplicates[mainWithDuplicates.main=="game development"])
display(mainWithDuplicates[mainWithDuplicates.duplicates=="game development"])

,duplicates,score,index,main,ind
189,html5 game development,95.0,1678,game development,development development game game html5


,duplicates,score,index,main,ind


In [9]:
# Similar way, "html5 game development" should be duplicates, not in main
display(mainWithDuplicates[mainWithDuplicates.main=="html5 game development"])
display(mainWithDuplicates[mainWithDuplicates.duplicates=="html5 game development"])

,duplicates,score,index,main,ind


,duplicates,score,index,main,ind
189,html5 game development,95.0,1678,game development,development development game game html5


In [10]:
# Sort the main df to find main courses. Since main course should have higher sale than duplicates
sortDes = df.sort_values("unitSold", ascending=False).drop_duplicates("courseTitle")

# Sort(ascending order) the main df to find main courses. Since dup courses should have lesser sale than main
sortAsc = df.sort_values("unitSold").drop_duplicates("courseTitle")

In [11]:
# Retrive the main courses with id
mainCourseWithId = pd.merge(mainWithDuplicates, sortDes, left_on="main", right_on="courseTitle")\
[["duplicates", "score", "main", "courseId"]].rename(columns={"courseId":"mainId"})

# Retrive the duplicates with id
dupCourseWithId = pd.merge(mainWithDuplicates, sortAsc, left_on="duplicates", right_on="courseTitle")\
[["duplicates", "score", "main", "courseId"]].rename(columns={"courseId":"dupId"})

In [12]:
# Merge the 2 dfs to get main and duplicates id together
merged = pd.merge(mainCourseWithId.drop_duplicates("duplicates"), dupCourseWithId.drop_duplicates("duplicates"), on="duplicates")

# Rename required columns
merged = merged[["main_x", "mainId", "duplicates", "dupId", "score_x"]]
merged.columns = merged.columns.str.replace("_x", "")
merged.head()

,main,mainId,duplicates,dupId,score
0,mental health,282714,youth mental health,297959,95.0
1,property development,239198,uk property development & law,284515,95.0
2,uk employment law,233696,employment law,293962,95.0
3,criminology and profiling,259499,criminology & profiling,247445,95.0
4,tax accounting,282917,uk tax accounting,284297,95.0


In [13]:
# This is our final data after find duplicates with main courses using fuzzywuzzy
fuzzyFinal = pd.merge(mainCourseWithId, merged, on="duplicates")[["main_x", "mainId_x", "duplicates", "dupId", "score_x"]]
fuzzyFinal.columns = fuzzyFinal.columns.str.replace("_x", "")
fuzzyFinal.head()

,main,mainId,duplicates,dupId,score
0,mental health,282714,youth mental health,297959,95.0
1,property development,239198,uk property development & law,284515,95.0
2,uk employment law,233696,employment law,293962,95.0
3,criminology and profiling,259499,criminology & profiling,247445,95.0
4,tax accounting,282917,uk tax accounting,284297,95.0


In [14]:
# Find exact duplicates with main courses
def findExactDuplicates(text):
    """text = text for which we will find duplicates"""
    
    # Sort the main df
    sortedDf = df[df.courseTitle==text].sort_values("unitSold", ascending=False)
    
    # Main course with id
    mainTitle = sortedDf.iloc[[0]].courseTitle.values
    mainId = sortedDf.iloc[[0]].courseId.values
    
    # Duplicate course with id
    dupTitle = sortedDf.iloc[1:].courseTitle.values
    dupId = sortedDf.iloc[1:].courseId.values
    
    # Create a df
    exactDup = pd.DataFrame({
        "duplicates":dupTitle,
        "dupId":dupId
    })
    
    # Insert main course and id
    exactDup["main"] = mainTitle[0]
    exactDup["mainId"] = mainId[0]
    
    # Insert score
    exactDup["score"] = 100
    return exactDup

In [15]:
# Apply the function to find duplicates(exact) with main
exactDupDf = pd.concat(list(map(findExactDuplicates, dfDup.courseTitle.unique()))).reset_index(drop=True)

In [16]:
# Merge fuzzywuzzy and exact duplicates together
finalDf = pd.concat([fuzzyFinal, exactDupDf])
finalDf.head()

,main,mainId,duplicates,dupId,score
0,mental health,282714,youth mental health,297959,95.0
1,property development,239198,uk property development & law,284515,95.0
2,uk employment law,233696,employment law,293962,95.0
3,criminology and profiling,259499,criminology & profiling,247445,95.0
4,tax accounting,282917,uk tax accounting,284297,95.0


In [17]:
# Create a dataframe to map mainid against all the duplicate ids
def createDict(text):
    """text = text for which we're finding main id"""
    
    # Create dictionary of ids
    idDict = {
        # Main of of a course
        "main":finalDf[finalDf.main==text].mainId.unique()[0],
        # Corresponding duplicate ids
        "dup":finalDf[finalDf.main==text].dupId.values
    }
    
    # Create a dataframe
    df = pd.DataFrame(idDict.items())
    
    # Set column 0 as index
    df = df.set_index(0)
    
    # Transpose the dataframe
    return df.T

In [18]:
# Apply the function to find main id with its all duplicate ids
mainVsDup = pd.concat(list(map(createDict, finalDf.main.unique()))).reset_index(drop=True)
mainVsDup.head()

,main,dup
0,282714,"[297959, 266953]"
1,239198,"[284515, 234897]"
2,233696,[293962]
3,259499,[247445]
4,282917,"[284297, 299208, 239181]"
